In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
import cv2
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D,Activation,Dropout
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import numpy as np 
import pandas as pd 



In [4]:
mapping={"paper":0,"rock":1,"scissors":2}
reverse_mapping = {0:"paper",1:"rock",2:"scissors"}

In [5]:
directory = '/kaggle/input/rock-paper-scissors-dataset'
print(os.listdir(directory))

In [6]:
dataset = []
for partition in ['train','test','val']:
    for el in ['paper','rock','scissors']:
        for file in os.listdir(os.path.join(directory,partition,el)):
            image = load_img(os.path.join(directory,partition,el,file),target_size = (200,200,3))
            image = img_to_array(image)
            image = tf.keras.applications.vgg16.preprocess_input(image)
            #image = image/255.0
            dataset.append([image,mapping[el]])
        
    
        
        
    
    

In [7]:
data,labels = zip(*dataset)

In [8]:
print(labels[0])

In [9]:
plt.imshow(data[0])

In [10]:
data[0].shape

In [11]:
print(len(labels))

In [12]:
r = 0
p = 0
s = 0
for i in labels:
    if i == 0:
        p+=1
    elif i==1:
        r+=1
    elif i ==2:
        s +=1
    else:
        print("mistake")
print("Number of images of rock: {}\nNumber of images of paper: {}\nNumber of images of scissors: {}".format(p,r,s))

In [13]:
labels=to_categorical(labels)
data=np.array(data)
labels=np.array(labels)
print("Data Shape:{}\nLabels shape: {}".format(data.shape,labels.shape))

In [14]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,shuffle = True)

In [15]:
trainx,valx,trainy,valy = train_test_split(trainx,trainy,test_size = 0.2,shuffle = True)

In [16]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=20,
                             zoom_range=0.2,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             shear_range=0.1,
                             fill_mode="nearest")

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(input_shape = (200,200,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )

In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2048,activation = 'relu'),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dense(3,activation = 'relu')
    
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=16),validation_data=(valx,valy),epochs=40)

In [ ]:
model.save('modelvgg3.h5')

In [ ]:
plt.imshow(testx[270])

In [ ]:
print(reverse_mapping[np.argmax(model.predict(testx[270].reshape(-1,200,200,3)))])

In [ ]:
model = create_model()

In [ ]:
os.listdir('/kaggle/working')

In [17]:
model = tf.keras.models.load_model('../input/modelvgg/model2.h5')


In [18]:
model

In [19]:
for i in range(4):
    img = testx[i]
    pred = model.predict(img.reshape(-1,200,200,3))
    pred = reverse_mapping[np.argmax(pred)]
    tru = testy[i]
    plt.imshow(img)
    print("Prediction {}, tru value: {}".format(pred,tru))

In [26]:
plt.imshow(testx[200])

In [28]:
print(reverse_mapping[np.argmax(model.predict(testx[200].reshape(-1,200,200,3)))])

In [53]:
pred = model.predict(testx)

In [54]:
cnt = 0

In [55]:
for p, t in zip(pred,testy):
    prediction = np.argmax(p)
    tes = np.argmax(t)
    if(tes==prediction):
        cnt+=1
    

In [56]:
cnt/pred.shape[0]*100